In [2]:
import argparse
import os
import numpy as np
from pysdf import SDF
import trimesh
from tqdm.notebook import tqdm
from einops import rearrange, reduce

In [3]:
parser = argparse.ArgumentParser()

# parser.add_argument('--path', type=str, default='./incode_data/shape/lucy.ply',
#                     help='Path to the object to reconstruct')
parser.add_argument('--path', type=str, default='/run/media/elliot/joint/courses/ra/ali_morseli/inr/data/Shape/Armadillo.ply',
                    help='Path to the object to reconstruct')
# parser.add_argument('--output_path', type=str, default='./incode_data/shape/',
#                     help='Output path')
parser.add_argument('--output_path', type=str, default='/run/media/elliot/joint/courses/ra/ali_morseli/inr/data/Shape/shape_npy2/',
                    help='Output path')
parser.add_argument('--N', type=int, default=512,
                    help='Resolution (N^3) of the mesh, same for xyz')
parser.add_argument('--M', type=int, default=1,
                    help='''Number of samples inside each cell to predict
                    gt occupancy value. Larger value yields more precise result.
                    Must be a ODD CUBIC number (M=1, with larger T is also fine).
                    ''')
parser.add_argument('--T', type=int, default=5,
                    help='''For complex mesh (typically non-watertight),
                    infer sdf multiple times and take the average.
                    Must be a ODD number (around 5~9 is enough).
                    ''')
args = parser.parse_args(args=[])

## Preprocessing

In [4]:
# From https://github.com/kwea123/MINER_pl/blob/master/preprocess_mesh.py
def volume_preprocessing(N, M, T):
    assert M%2 == 1, 'M must be an odd cubic number!!'
    assert T%2 == 1, 'T must be an odd number!!'
    cbrtM = int(M**(1/3))

    o = trimesh.load(args.path, force='mesh', skip_materials=True)
    bbox = np.amax(o.vertices, 0)-np.amin(o.vertices, 0)
    mesh_whl = bbox/2
    o.vertices -= np.amax(o.vertices, 0)-mesh_whl # center the mesh
    mesh_whl *= 1.02 # give some margin
    xs = np.linspace(-mesh_whl[0], mesh_whl[0], cbrtM*N)
    ys = np.linspace(-mesh_whl[1], mesh_whl[1], cbrtM*N)
    zs = np.linspace(-mesh_whl[2], mesh_whl[2], cbrtM*N)
    occ = np.zeros((N, N, N, 1), np.float32)

    print('Computing occupancy values ...')
    for t in tqdm(range(T)):
        f = SDF(o.vertices, o.faces) # the sdf is different each time...
        for i, z in enumerate(tqdm(zs[::cbrtM])):
            xyz_ = np.stack(np.meshgrid(xs, ys, zs[i*cbrtM:(i+1)*cbrtM]), -1).reshape(-1, 3)
            occ_ = f.contains(xyz_).reshape(cbrtM*N, cbrtM*N, cbrtM)
            occ_ = rearrange(occ_, '(h a) (w b) c -> (a b c) h w',
                             a=cbrtM, b=cbrtM, c=cbrtM, h=N, w=N).mean(0)
            occ[:, :, i, 0] += occ_.astype(np.float32)

    occ = (occ>T/2).astype(bool)
    
    return np.packbits(occ), mesh_whl

In [5]:
occ_im, mesh_whl = volume_preprocessing(args.N, args.M, args.T)
occ_gt_im, _ = volume_preprocessing(128, args.M, args.T)

Computing occupancy values ...


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/512 [00:00<?, ?it/s]

  0%|          | 0/512 [00:00<?, ?it/s]

  0%|          | 0/512 [00:00<?, ?it/s]

...

  0%|          | 0/128 [00:00<?, ?it/s]

  0%|          | 0/128 [00:00<?, ?it/s]

  0%|          | 0/128 [00:00<?, ?it/s]

  0%|          | 0/128 [00:00<?, ?it/s]

  0%|          | 0/128 [00:00<?, ?it/s]

In [6]:
data = {'im': occ_im, 'mesh_whl': mesh_whl, 'gt_im': occ_gt_im}

In [7]:
os.makedirs(args.output_path + 'occupancies', exist_ok=True)
base = os.path.basename(args.path)
save_path = args.output_path + f'occupancies/preprocessed_{os.path.splitext(base)[0]}.npy'
np.save(save_path, data)
print(f'Occupancy saved to {save_path} !')

Occupancy saved to /run/media/elliot/joint/courses/ra/ali_morseli/inr/data/Shape/shape_npy2/occupancies/preprocessed_Armadillo.npy !


In [8]:
# testing
occ_gt_im.shape

(262144,)